In [1]:
import pandas as pd
import gmaps
import requests
#import API key
from config import gkey

In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=gkey)

In [4]:
#Store the CSV you saved created in part one into a DataFrame
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Saint-Philippe,RE,2021-05-20 03:53:48,-21.3585,55.7679,72.99,81,97,21.07
1,1,Vyshhorod,UA,2021-05-20 03:53:48,50.5848,30.4898,52.93,93,100,15.19
2,2,Faya,SA,2021-05-20 03:53:48,18.3851,42.4509,64.92,25,10,4.38
3,3,La Ronge,CA,2021-05-20 03:53:49,55.1001,-105.2842,28.65,54,90,11.50
4,4,Mahebourg,MU,2021-05-20 03:53:49,-20.4081,57.7000,74.25,75,86,25.86


In [5]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [19]:
#1. Assign the locations to an array of latitude and longitude pairs
locations = city_data_df[["Lat","Lng"]]

#2. Assign the weights variable to some values (add for loop if max temp less than 0)
max_temp = city_data_df["Max Temp"]
#temps =[]
#for temp in max_temp:
    #temps.append(max(temp,0))

#3. Assign the figure variable to the gmaps.figure() attribute
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)

#4. Assign the heatmap_layer variabel to the heatmap_layer attribute and add in the location
#instead of using for loop above, can use list comprehension
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating = False, max_intensity = 300, point_radius = 4)



#5. Add the heatmap layer 
fig.add_layer(heatmap_layer)

#6. call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
#heat map of precent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30,31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating = False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
#heat map of precent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30,31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating = False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
#heat map of precent cloudiness
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30,31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating = False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
#ask the customer to add a min and max temp value
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [27]:
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<=max_temp) & 
                                       (city_data_df["Max Temp"]>=min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
5,5,Denham Springs,US,2021-05-20 03:49:14,30.4869,-90.9562,75.06,95,90,1.99
9,9,Kapaa,US,2021-05-20 03:53:50,22.0752,-159.3190,78.78,69,93,5.01
10,10,Ulaanbaatar,MN,2021-05-20 03:49:03,47.9077,106.8832,75.33,23,0,15.70
14,14,Hithadhoo,MV,2021-05-20 03:53:52,-0.6000,73.0833,84.25,70,18,8.52
22,22,East London,ZA,2021-05-20 03:53:54,-33.0153,27.9116,83.34,26,0,11.50


In [28]:
preferred_cities_df.count()

City_ID       164
City          164
Country       164
Date          164
Lat           164
Lng           164
Max Temp      164
Humidity      164
Cloudiness    164
Wind Speed    164
dtype: int64

In [31]:
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Denham Springs,US,75.06,30.4869,-90.9562,
9,Kapaa,US,78.78,22.0752,-159.3190,
10,Ulaanbaatar,MN,75.33,47.9077,106.8832,
14,Hithadhoo,MV,84.25,-0.6000,73.0833,
22,East London,ZA,83.34,-33.0153,27.9116,


In [44]:
#set parameters to search for a hotel
params = {
    "radius":5000,
    "type":"lodging",
    "key":gkey}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found, skipping")
print(f'-------------------')
print(f'Done retrieving data')
print(f'-------------------')

Hotel not found, skipping
Hotel not found, skipping
Hotel not found, skipping
Hotel not found, skipping
Hotel not found, skipping
Hotel not found, skipping
Hotel not found, skipping
Hotel not found, skipping
Hotel not found, skipping
Hotel not found, skipping
Hotel not found, skipping
-------------------
Done retrieving data
-------------------


In [45]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Denham Springs,US,75.06,30.4869,-90.9562,"Candlewood Suites Denham Springs, an IHG Hotel"
9,Kapaa,US,78.78,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
10,Ulaanbaatar,MN,75.33,47.9077,106.8832,The Blue Sky Hotel and Tower
14,Hithadhoo,MV,84.25,-0.6000,73.0833,Scoop Guest House
22,East London,ZA,83.34,-33.0153,27.9116,Tu Casa
36,Beirut,LB,82.38,33.8889,35.4944,"Crowne Plaza Beirut, an IHG Hotel"
38,Sola,VU,81.27,-13.8833,167.5500,Leumerus Bungalows
40,Lagos,NG,80.92,6.5833,3.7500,
46,Hilo,US,76.28,19.7297,-155.0900,Hilo Hawaiian Hotel
55,Moranbah,AU,78.51,-22.0016,148.0466,Direct Hotels - Monterey Moranbah


In [1]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

#add a heatmap of temperature for the vacation spots
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30,31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

NameError: name 'hotel_df' is not defined